In [2]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [3]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, \
    MultiThreadSLIM_SLIMElasticNetRecommender

slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

item = ItemKNNCFRecommender(URM_train)

rp3beta = RP3betaRecommender(URM_train)

slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers = 7)
item.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
rp3beta.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 483 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 358 ( 1.6%) items with no interactions.
ItemKNNCFRecommender: URM Detected 483 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 358 ( 1.6%) items with no interactions.
RP3betaRecommender: URM Detected 483 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 358 ( 1.6%) items with no interactions.


100%|█████████▉| 22192/22222 [04:00<00:00, 120.76it/s]

Similarity column 22222 (100.0%), 8863.88 column/sec. Elapsed time 2.51 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 4285.09 column/sec. Elapsed time 5.19 sec
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [4]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
import optuna as op
model = ItemKNNCustomSimilarityRecommender

def objective(trial):
    alpha = trial.suggest_float("alpha", 5e-4, 1)
    beta = trial.suggest_float("beta", 5e-4, 1)

    new_similarity = (1 - alpha - beta) * slim.W_sparse + alpha * item.W_sparse + beta * rp3beta.W_sparse
    recommender = model(URM_train)
    recommender.fit(W_sparse=new_similarity)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

100%|█████████▉| 22216/22222 [04:40<00:00, 79.18it/s] 


In [5]:
best_params = {
}
study_name = "cust-sim-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

/tmp/ipykernel_193094/3446642297.py:5: FutureWarning: delete_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  op.delete_study(study_name, storage_name)
[I 2023-12-09 17:27:04,936] A new study created in RDB with name: cust-sim-study


ItemKNNCustomSimilarityRecommender: URM Detected 483 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 358 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10024 (100.0%) in 7.32 sec. Users per second: 1369


[I 2023-12-09 17:27:12,391] Trial 0 finished with value: 0.027289212239830853 and parameters: {'alpha': 0.37880472880191335, 'beta': 0.36539122598158535}. Best is trial 0 with value: 0.027289212239830853.


ItemKNNCustomSimilarityRecommender: URM Detected 483 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 358 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10024 (100.0%) in 7.19 sec. Users per second: 1395


[I 2023-12-09 17:27:19,708] Trial 1 finished with value: 0.022185136624482203 and parameters: {'alpha': 0.7243297217165667, 'beta': 0.4119240114293423}. Best is trial 0 with value: 0.027289212239830853.


...

[I 2023-12-09 17:38:11,563] Trial 97 finished with value: 0.030376311138980927 and parameters: {'alpha': 0.03942682611035651, 'beta': 0.6479476699558064}. Best is trial 63 with value: 0.030517753423529817.


ItemKNNCustomSimilarityRecommender: URM Detected 483 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 358 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10024 (100.0%) in 6.78 sec. Users per second: 1479


[I 2023-12-09 17:38:18,470] Trial 98 finished with value: 0.02987847024284596 and parameters: {'alpha': 0.09505308486222877, 'beta': 0.7014887379719245}. Best is trial 63 with value: 0.030517753423529817.


ItemKNNCustomSimilarityRecommender: URM Detected 483 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 358 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10024 (100.0%) in 6.69 sec. Users per second: 1498


[I 2023-12-09 17:38:25,274] Trial 99 finished with value: 0.030470846159693234 and parameters: {'alpha': 0.018462594680569173, 'beta': 0.6129803468459561}. Best is trial 63 with value: 0.030517753423529817.


In [7]:
study.best_params

{'alpha': 0.10204960939360941}

In [10]:
new_similarity = (1 - 0.10204960939360941) * slim.W_sparse + 0.10204960939360941 * item.W_sparse
recommender = model(URM_train)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
recommender.fit(W_sparse=new_similarity)
evaluator_test.evaluateRecommender(recommender)

ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Ignoring 2137 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10501 (100.0%) in 6.86 sec. Users per second: 1532


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.078231                 0.137502  0.117368  0.037575    0.066342   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.229908  0.123998  0.093884  0.466908      0.292136  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.830907          0.387957    0.830907        0.04454   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            9.767962                    0.99654             0.129436   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Ignoring 2137 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10501 (100.0%) in 6.83 sec. Users per second: 1538


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.077831                 0.135528  0.115191  0.037798    0.066234   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.231465  0.123572  0.092895  0.464146      0.293879  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.830907          0.385662    0.830907       0.050025   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.028651                   0.997428             0.145375   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
## MAP_MIN_DEN = .0822900 ## MAP = 0.0483066 the best 0.139

In [ ]:
## MAP_MIN_DEN = .0845288 ## MAP = 0.0495724 second 0.135

In [ ]:
## MAP_MIN_DEN = .0846506 ## MAP = 0.0496060 third 